In [1]:
import datetime, time, psutil, os
import numpy as np
import pandas as pd
import nltk
from sklearn.feature_extraction import stop_words
#from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

from sklearn.naive_bayes import GaussianNB,MultinomialNB,ComplementNB,BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
print(str(datetime.datetime.now()) + "   start of baseline model")
#  CONTROL PARAMTERS
size = 10
tfTransform = True
filename = "GB_SIZE_FEATURE_"+str(size)+"_"+str(int(time.time()))+".txt"
print(filename)
f = open(filename, "w")
f.write('environment \n')
f.write(str(platform.uname()) + '\n')
f.write(str(psutil.virtual_memory()) + '\n')
f.write(str(psutil.disk_partitions()) + '\n\n')
f.flush()

2019-04-11 23:06:18.419254   start of baseline model
GB_SIZE_FEATURE_10_1555049178.txt


In [3]:
f.write( 'control parameter: size {} , tf transform {}\n\n'.format(size, tfTransform))

48

In [4]:
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

f.write("train_data shape: "+str(train_data.shape)+ '\n')
f.write("test_data shape: "+str(test_data.shape)+ '\n\n')

30

In [5]:
train_data = train_data[:len(train_data)//size] 
test_data = test_data[:len(test_data)//size] 

f.write("applied train_data shape: "+str(train_data.shape)+ '\n')
f.write("applied test_data shape: "+str(test_data.shape)+ '\n\n')
f.flush()
train_data.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [6]:
train_data['target'].value_counts()

0    122567
1      8045
Name: target, dtype: int64

In [7]:
train_data['num_words'] = train_data['question_text'].apply(lambda x: len(str(x).split()) )

In [8]:
train_data['num_words'].describe()

count    130612.000000
mean         12.774462
std           7.026658
min           1.000000
25%           8.000000
50%          11.000000
75%          15.000000
max          65.000000
Name: num_words, dtype: float64

In [9]:
start_time = time.time()

train_text = train_data['question_text']
test_text = test_data['question_text']
train_target = train_data['target']
all_text = train_text.append(test_text)

#count_vectorizer = CountVectorizer(stop_words=stop_words.ENGLISH_STOP_WORDS)
count_vectorizer = CountVectorizer()
count_vectorizer.fit(all_text)

elapsed_time = time.time() - start_time
print(str(time.strftime("%H:%M:%S:%m", time.gmtime(elapsed_time))))
f.write(str(time.strftime("%H:%M:%S:%m", time.gmtime(elapsed_time))) + "   vectorization\n")
f.flush()

00:00:02:01


In [10]:
start_time = time.time()

train_text_features_cv = count_vectorizer.transform(train_text)
test_text_features_cv = count_vectorizer.transform(test_text)

print(train_text_features_cv.get_shape())
print(test_text_features_cv.get_shape())

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(all_text)
train_text_features_tf = tfidf_vectorizer.transform(train_text)
test_text_features_tf = tfidf_vectorizer.transform(test_text)

print(train_text_features_tf.get_shape())
print(test_text_features_tf.get_shape())

tfTransform = True
if tfTransform :
    train_text_features = train_text_features_tf
    test_text_features = test_text_features_tf
else:
    train_text_features = train_text_features_cv
    test_text_features = test_text_features_cv

    
elapsed_time = time.time() - start_time
print(train_text_features.get_shape())
print(test_text_features.get_shape())
print(str(time.strftime("%H:%M:%S:%m", time.gmtime(elapsed_time))) + "   feature transform")

f.write("train_text_features_cv" +str(train_text_features_cv.get_shape())+"\n")
f.write("test_text_features_cv" +str(test_text_features_cv.get_shape())+"\n")
f.write("train_text_features_tf" +str(train_text_features_tf.get_shape())+"\n")
f.write("test_text_features_tf" +str(test_text_features_tf.get_shape())+"\n")
f.write("train_text_features" +str(train_text_features.get_shape())+"\n")
f.write("test_text_features" +str(test_text_features.get_shape())+"\n")
f.write(str(time.strftime("%H:%M:%S:%m", time.gmtime(elapsed_time))) + "   feature transform\n")
f.flush()

(130612, 67139)
(37580, 67139)
(130612, 67139)
(37580, 67139)
(130612, 67139)
(37580, 67139)
00:00:07:01   feature transform


In [11]:
len(np.zeros([train_data.shape[0],]))

130612

In [12]:
#BernoulliNB
start_time = time.time()

kfold = KFold(n_splits = 5, shuffle = True, random_state = 1)

test_preds = 0
oof_preds = np.zeros([train_data.shape[0],])

for i, (train_idx,valid_idx) in enumerate(kfold.split(train_data)):
    x_train, x_valid = train_text_features[train_idx,:], train_text_features[valid_idx,:]
    y_train, y_valid = train_target[train_idx], train_target[valid_idx]
    classifier = BernoulliNB()
    classifier.fit(x_train,y_train)
   
    oof_preds[valid_idx] = classifier.predict_proba(x_valid)[:,1]
    test_preds += 0.2*classifier.predict_proba(test_text_features)[:,1]

pred_train = (oof_preds > 0.25).astype(np.int)


elapsed_time = time.time() - start_time
print(str(time.strftime("%H:%M:%S:%m", time.gmtime(elapsed_time))) + "   BernoulliNB")
print("f1_score: " +str(f1_score(train_target, pred_train)))
f.write(str(time.strftime("%H:%M:%S:%m", time.gmtime(elapsed_time))) + "   BernoulliNB\n")
f.write("f1_score: " +str(f1_score(train_target, pred_train)) +"\n")
f.flush()

00:00:00:01   BernoulliNB
f1_score: 0.42209844949570974


In [13]:
#LogisticRegression
start_time = time.time()

kfold = KFold(n_splits = 5, shuffle = True, random_state = 1)

test_preds = 0
oof_preds = np.zeros([train_data.shape[0],])

for i, (train_idx,valid_idx) in enumerate(kfold.split(train_data)):
    x_train, x_valid = train_text_features[train_idx,:], train_text_features[valid_idx,:]
    y_train, y_valid = train_target[train_idx], train_target[valid_idx]
    classifier = LogisticRegression(solver='liblinear')
    classifier.fit(x_train,y_train)
   
    oof_preds[valid_idx] = classifier.predict_proba(x_valid)[:,1]
    test_preds += 0.2*classifier.predict_proba(test_text_features)[:,1]

pred_train = (oof_preds > 0.25).astype(np.int)


elapsed_time = time.time() - start_time
print(str(time.strftime("%H:%M:%S:%m", time.gmtime(elapsed_time))) + "   LogisticRegression")
print("f1_score: " +str(f1_score(train_target, pred_train)))
f.write(str(time.strftime("%H:%M:%S:%m", time.gmtime(elapsed_time))) + "   LogisticRegression\n")
f.write("f1_score: " +str(f1_score(train_target, pred_train)) +"\n")
f.flush()

00:00:02:01   LogisticRegression
f1_score: 0.5603949259513841


In [14]:
#GradientBoostingClassifier
start_time = time.time()

kfold = KFold(n_splits = 5, shuffle = True, random_state = 1)

test_preds = 0
oof_preds = np.zeros([train_data.shape[0],])

for i, (train_idx,valid_idx) in enumerate(kfold.split(train_data)):
    x_train, x_valid = train_text_features[train_idx,:], train_text_features[valid_idx,:]
    y_train, y_valid = train_target[train_idx], train_target[valid_idx]
    classifier = GradientBoostingClassifier(random_state=1)
    classifier.fit(x_train,y_train)
   
    oof_preds[valid_idx] = classifier.predict_proba(x_valid)[:,1]
    test_preds += 0.2*classifier.predict_proba(test_text_features)[:,1]

pred_train = (oof_preds > 0.25).astype(np.int)


elapsed_time = time.time() - start_time
print(str(time.strftime("%H:%M:%S:%m", time.gmtime(elapsed_time))) + "   GradientBoostingClassifier")
print("f1_score: " +str(f1_score(train_target, pred_train)))
f.write(str(time.strftime("%H:%M:%S:%m", time.gmtime(elapsed_time))) + "   GradientBoostingClassifier\n")
f.write("f1_score: " +str(f1_score(train_target, pred_train)) +"\n")
f.flush()

01:02:47:01   GradientBoostingClassifier
f1_score: 0.485589519650655


In [15]:
print(str(datetime.datetime.now())+ "   end of baseline model")
f.write(str(datetime.datetime.now())+ "   end of baseline model\n\n")

2019-04-12 00:09:22.958536   end of baseline model


52

In [16]:
f.close()